In [ ]:
# Import library
import requests 
import pandas as pd

from datetime import date
from datetime import timedelta

import datetime
import time

# Filter Column we need

In [ ]:
sheet_df    = pd.read_csv(gsheet_url)

print('We need these columns:')
print()
sheet_df['Column']

In [ ]:
df = pd.read_pickle(r'FILE')

# Take only columns we need:
df = df[sheet_df['Column']]
df.head(5)

# Create more column

## Create column from raw data

In [ ]:
# Create Revenue column: Revenue = COD + bank transfer
df['revenue'] = df['cod'] + df['transfer_money']

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# Creat tag column
tag_list = []
for i in df['order_tags']:
    tag = []
    for q in i:
        if str(q).find("'value_name':") == -1:
            tag.append('')
        else:
            j = 'Tag: (' + q['value_name'] + ')'
            tag.append(j)
    k = ', '.join(tag)
    tag_list.append(k)
df['order_tags'] = tag_list

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# Create Warehouse column
warehouse = []
for i in df['warehouse_name']:
    if type(i) == dict and 'value_name' in i:
        warehouse.append(i['value_name'])
    else:
        warehouse.append('')
df['warehouse_name'] = warehouse

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# create seller column:
seller = []
for i in df['employees']:
    if type(i) == dict and 'value_name' in i: 
        seller.append(i['value_name'])
    else:
        seller.append('')
    
df['employees'] = seller

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# create courier partner_shipping column
partner_shipping = []
for i in df['partner_shipping']:
    if str(i).find('extend_code') != -1 and str(i).find('order_number_vtp') != -1: 
        j = str(i['extend_code']) + ' ' + str(i['order_number_vtp'])
        partner_shipping.append(j)
    else:
        partner_shipping.append('')
df['partner_shipping'] = partner_shipping

## Transform these columns:

In [ ]:
df2 = df

def convert_time(x):
    convert_time = []
    for i in x:
        if i != '':
            i = i.split('T', 1)[0] +' '+ i.split('T', 1)[1]
            i = pd.to_datetime(
            
                    i[:19]
            
                ) + pd.DateOffset(hours=7)
            convert_time.append(i)
        else:
            convert_time.append('')
    return convert_time

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# funtion to calculate time duration':
def duration(end_time, start_time, unit):
    duration_list = []
    for i, e in zip(end_time, start_time):
        
        if i != '' and e != '':
            duration = pd.to_datetime(i) - pd.to_datetime(e)
            if unit    == 'day':
                duration_list.append(round(duration / pd.Timedelta(days=1), 1) )
                
            elif unit  == 'hour':
                duration_list.append(round(duration / pd.Timedelta(hours=1), 1) )
                
            elif unit  == 'minute':
                duration_list.append(round(duration / pd.Timedelta(minutes=1), 1) )
        else:
            duration_list.append('')
    return duration_list

In [ ]:
# timeline status:
status_timeline = []
for i in df2['status_timeline']:
    if str(i).find("'status':") == -1:
        status_timeline.append('')
        
    else:
        status = []
        for j in i:
            a  = j['updated_at'].split('T', 1)[0] +' '+ j['updated_at'].split('T', 1)[1]
            updated_at = pd.to_datetime(a[:19]) + pd.DateOffset(hours=7)
            k  = 'S: ' + str(j['status']) + ' - ' + str(updated_at)
            status.append(k)
        z = ', '.join(status)
        if len(z) < 1000:
            status_timeline.append(z)
        else:
            status_timeline.append('')
df2['status_timeline'] = status_timeline

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# convert time:
df2['inserted_at']                = convert_time(df2['inserted_at'])
df2['updated_at']                 = convert_time(df2['updated_at'])

In [ ]:
# Delete no need columns:
df2 = df2.drop(columns = ['S', 'O', 'cod', 'transfer_money'])

df2.info()

# Gadget (Chunk list + trigger)

In [ ]:
# Create a CSV file
df2.to_csv(r'FILE')

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# Create the pandas DataFrame with column value_name is provided explicitly
number_of_chunk = len(df2) / 100

chunk_list = pd.DataFrame(list(range(0, int(number_of_chunk)+1)), columns=['Chunk'])
chunk_list.to_csv(r'FILE')
# print dataframe.
chunk_list

# ______________________________________________________________________________________________________________________________________________________________________________________________________

# Create trigger
trigger = pd.DataFrame()
# Printing Empty DataFrame
trigger.to_csv(r'FILE')